In [1]:
!nvidia-smi

Wed Dec 15 13:47:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install Arabic-Stopwords
!pip install arabic_reshaper
!pip install python-bidi
!pip install tensorflow-addons #to use f1 score in complie's metrics

     |████████████████████████████████| 353 kB 5.3 MB/s 
     |████████████████████████████████| 126 kB 33.0 MB/s 
     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM
import tensorflow_addons as tfa
from numpy import array
import pandas as pd
import gensim
from gensim.models import KeyedVectors
from gensim.models import word2vec

**Large Multi-Domain Resources for Arabic Sentiment Analysis**

For this dataset in MSA, I will use w2v pretrained on Wikipedia.

In [4]:
#read files of first dataset (MSA)
res_df=pd.read_csv('https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/master/datasets/RES.csv')
prod_df=pd.read_csv('https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/master/datasets/PROD.csv')
htl_df=pd.read_csv('https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/master/datasets/HTL.csv')
mov_df=pd.read_csv('https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/master/datasets/MOV.csv')

In [5]:
#keep only binary classes (pos & neg)
res_df=res_df[res_df['polarity']!=0].reset_index(drop=True)
prod_df=prod_df[prod_df['polarity']!=0].reset_index(drop=True)
htl_df=htl_df[htl_df['polarity']!=0].reset_index(drop=True)
mov_df=mov_df[mov_df['polarity']!=0].reset_index(drop=True)

In [6]:
datasetDict = {"resturants": res_df, "products": prod_df, "hotels": htl_df, "movies": mov_df}

In [12]:
# convert neg label from -1 to 0
for k, v in datasetDict.items():
  v['polarity'].replace({-1: 0}, inplace=True)

In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import string
import arabicstopwords.arabicstopwords as ar_words

ar_sw=['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'إنها', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما','كان','كانت', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', 'قد', 'كأن', 'كأنما', 'كأي', 'كأين', 'كذا', 'كذلك', 'كل', 'كلا', 'كلاهما', 'كلتا', 'كلما', 'كليكما', 'كليهما', 'كم', 'كم', 'كما', 'كي', 'كيت', 'كيف', 'كيفما', 'لا', 'لاسيما', 'لدى', 'لست', 'لستم', 'لستما', 'لستن', 'لسن', 'لسنا', 'لعل', 'لك', 'لكم', 'لكما', 'لكن', 'لكنما', 'لكي', 'لكيلا', 'لم', 'لما', 'لن', 'لنا', 'له', 'لها', 'لهم', 'لهما', 'لهن', 'لو', 'لولا', 'لوما', 'لي', 'لئن', 'ليت', 'ليس', 'ليسا', 'ليست', 'ليستا', 'ليسوا', 'ما', 'ماذا', 'متى', 'مذ', 'مع', 'مما', 'ممن', 'من', 'منه', 'منها', 'منذ', 'مه', 'مهما', 'نحن', 'نحو', 'نعم', 'ها', 'هاتان', 'هاته', 'هاتي', 'هاتين', 'هاك', 'هاهنا', 'هذا', 'هذان', 'هذه', 'هذي', 'هذين', 'هكذا', 'هل', 'هلا', 'هم', 'هما', 'هن', 'هنا', 'هناك', 'هنالك', 'هو', 'هؤلاء', 'هي', 'هيا', 'هيت', 'هيهات', 'والذي', 'والذين', 'وإذ', 'وإذا', 'وإن', 'ولا', 'ولكن', 'ولو', 'وما', 'ومن', 'وهو', 'يا']

def normalizeArabic(t):
    t = re.sub("[إأٱآا]", "ا", t)
    t = re.sub("ى", "ي", t)
    t = re.sub("ة", 'ه', t)
    t = re.sub("ؤ", "ء", t)
    t = re.sub("ئ", "ء", t)
    return (t)

ar_stop= []
for w in ar_sw:
  ar_stop.append(normalizeArabic(w))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
#COLLECT Ar stopwords from 2 sources

ar_stopwords = stopwords.words('arabic') + list(ar_words.stopwords_list()) + ar_stop
print('nlkt arabic stopwords =',len(stopwords.words('arabic')))
print('Arabic-Stopwords =', len(ar_words.stopwords_list()))
print('My list =', len(ar_stop))
print('sum =',len(ar_stopwords), 'unique=', len(set(ar_stopwords)) )

nlkt arabic stopwords = 754
Arabic-Stopwords = 13629
My list = 251
sum = 14634 unique= 13997


In [9]:
# define 'clean_tweet' function to clean the text and remove unwanted text parts
def clean_text(text):
    # define regular expression patterns
    p_english = "[a-zA-Z0-9]+"
    p_url = "https?://[A-Za-z0-9./]+"
    p_mention = "\@[\_0-9a-zA-Z]+\:?"    
    p_retweet = "RT \@[\_\-0-9a-zA-Z]+\:?"
    p_punctuations = "[" + string.punctuation + "]"
    
    # remove unwanted parts
    text = re.sub(p_english, ' ', text)
    text = re.sub(p_retweet, ' ', text)
    text = re.sub(p_mention, ' ', text)
    text = re.sub(p_url, ' ', text)
    text = re.sub(p_punctuations, ' ', text)
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    
    # remove الهمزة
    text = re.sub("[أإآ]", 'ا', text)
    text = re.sub("ة", 'ه', text)
    text = re.sub("ى", 'ي', text)
    
    # removing tashkeel
    tashkel = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(tashkel, '', text)
    
    # remove repeated letters more than two letters
    text = re.sub(r'(.)\1+', r'\1\1', text)
    
    #trim    
    text = text.strip()
    
    # remove stopwords
    words = [word for word in text.split() if word not in ar_stopwords]
    words = [word for word in words if len(word)>=2]
    
    # merge and return final text
    return ' '.join(words)

In [10]:
import re
import numpy as np
from nltk import ngrams

# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

In [11]:
# apply clean function on all text in the dataframes
for k, v in datasetDict.items():
    v['clean_text'] = v['text'].apply(clean_text)

In [13]:
!wget 'https://bakrianoo.ewr1.vultrobjects.com/aravec/full_uni_sg_300_wiki.zip'
!unzip 'full_uni_sg_300_wiki.zip'

--2021-12-15 14:19:10--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_uni_sg_300_wiki.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 720163266 (687M) [application/zip]
Saving to: ‘full_uni_sg_300_wiki.zip’

full_uni_sg_300_wik 100%[===================>] 686.80M  94.5MB/s    in 7.3s    

2021-12-15 14:19:18 (93.7 MB/s) - ‘full_uni_sg_300_wiki.zip’ saved [720163266/720163266]

Archive:  full_uni_sg_300_wiki.zip
  inflating: full_uni_sg_300_wiki.mdl  
  inflating: full_uni_sg_300_wiki.mdl.trainables.syn1neg.npy  
  inflating: full_uni_sg_300_wiki.mdl.wv.vectors.npy  


In [14]:
# load the AraVec model
w2v = gensim.models.Word2Vec.load("full_uni_sg_300_wiki.mdl")
print("We've",len(w2v.wv.index2word),"vocabularies") 

We've 320636 vocabularies


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

seq_len = 128 # standardized length of each word sequence 

#------------------------------------------------------ Choose a dataset df
df= res_df
#df= prod_df
#df= htl_df
#df= mov_df

#################################### vectorizing cleaned text
# fit tokenizer vocab 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.clean_text)

################################### standard train/val split
train_text, val_text, y_train, y_val = train_test_split(df.clean_text, df.polarity, 
                                                        test_size=0.2, random_state=123, stratify=df.polarity)


################################## indexing + padding
# convert train and val texts to token sequences of standardized length 128,
# padding fills leading 0s in or cuts off sequence at 128th word
train_text = tokenizer.texts_to_sequences(train_text) 
train_text = pad_sequences(train_text, maxlen=seq_len, padding='post')

val_text = tokenizer.texts_to_sequences(val_text)
val_text = pad_sequences(val_text, maxlen=seq_len, padding='post')

print('one training text after padding: ')
print(train_text[10])

################################### preparing embedding matrix
embedding_dim = w2v.vector_size # w2v embedding dim
word_index = tokenizer.word_index # vocab

# use the gensim model to build a numpy array of embeddings,
# we'll feed this array to the keras embeddings layer.
# each row i of the array will correspond to the word token assigned to that value 
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    try:
        embedding_vector = w2v[word]
        embedding_matrix[i] = embedding_vector
    except: # word in our data vocab is missing in w2v, will use 0 vector for that word
        pass
print('embedding matrix shape (vocab, embeddings dim) is ', embedding_matrix.shape)


##################################### defining model

inp = Input(shape=(seq_len,))
x = Embedding(len(word_index) + 1,
              embedding_dim,
              weights=[embedding_matrix], # where we feed the pretrained vecs
              trainable=False)(inp) # freeze these parameters in the model
#recurrent_dropout=.1
x = Bidirectional(LSTM(64, activation='relu'))(x)
x = Dense(32)(x) # fully connected layer on top of the output of the bi-LSTM
#x = Dropout(.3)(x)
#x = Dense(20)(x) #added
y = Dense(1, activation='sigmoid')(x)

NN = Model(inp, y)

print(NN.summary())

######################################## compiling model
threshold = 0.5
NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',tfa.metrics.F1Score(num_classes=2, average='micro', threshold=threshold)])


######################################## fitting model = training
NN.fit(train_text, y_train, validation_data=(val_text, y_val), epochs=10, batch_size=50, verbose=1)


######################################### testing

# This can be used if we have train, val, test sets
# but since data size is not to large, I choose 2 splits only
#result = the validation of last epoch

#prob= NN.predict(val_text)
#predections= [1 if p >= 0.5 else 0 for p in prob]
#f1_score(y_val, predections)


one training text after padding: 
[27250   150 10991    30    12   191   894   189   499   302  4058 27251
   409   183  2088   732   183  2892   145   326  1907 11482     9     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


#embedding matrix shape (vocab, embeddings dim) is  (46498, 300)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128)]             0         
                                                                 
 embedding (Embedding)       (None, 128, 300)          13949400  
                                                                 
 bidirectional (Bidirectiona  (None, 128)              186880    
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 14,140,441
Trainable params: 191,041
Non-trainab

In [18]:
prob= NN.predict(val_text)
predections= [1 if p >= 0.5 else 0 for p in prob]
print('---------------------------RESULT------------------------')
print('The model f1 score on validation set is :', f1_score(y_val, predections))

---------------------------RESULT------------------------
The model f1 score on validation set is : 0.8707482993197277


The next fit for the same above settings shows a bit better result! 

In [ ]:
#without dropout + clean text ------------ resturant
NN.fit(train_text, y_train, validation_data=(val_text, y_val), epochs=10, batch_size=50, verbose=1)

Epoch 1/10
172/172 [==============================] - 60s 331ms/step - loss: 0.5208 - acc: 0.7605 - f1_score: 0.8607 - val_loss: 0.4532 - val_acc: 0.7823 - val_f1_score: 0.8718
Epoch 2/10
172/172 [==============================] - 57s 331ms/step - loss: 0.3816 - acc: 0.8305 - f1_score: 0.8935 - val_loss: 0.3835 - val_acc: 0.8342 - val_f1_score: 0.8932
Epoch 3/10
172/172 [==============================] - 57s 329ms/step - loss: 4514.7544 - acc: 0.8512 - f1_score: 0.9047 - val_loss: 0.3753 - val_acc: 0.8417 - val_f1_score: 0.9006
Epoch 4/10
172/172 [==============================] - 57s 334ms/step - loss: 0.3134 - acc: 0.8663 - f1_score: 0.9141 - val_loss: 0.3673 - val_acc: 0.8389 - val_f1_score: 0.8974
Epoch 5/10
172/172 [==============================] - 57s 332ms/step - loss: 0.2940 - acc: 0.8761 - f1_score: 0.9201 - val_loss: 0.3754 - val_acc: 0.8375 - val_f1_score: 0.8918
Epoch 6/10
172/172 [==============================] - 57s 329ms/step - loss: 0.2720 - acc: 0.8859 - f1_score: 0.

.

Below are some past results with some different choices, for my own learning/comparing ..... 

In [ ]:
#with dropouts + clean text ------------ resturant
NN.fit(train_text, y_train, validation_data=(val_text, y_val), epochs=5, batch_size=50, verbose=1)

Epoch 1/5
172/172 [==============================] - 96s 513ms/step - loss: 1.1893 - acc: 0.7738 - f1_score: 0.8652 - val_loss: 0.4601 - val_acc: 0.7833 - val_f1_score: 0.8719
Epoch 2/5
172/172 [==============================] - 88s 510ms/step - loss: 0.4338 - acc: 0.7884 - f1_score: 0.8743 - val_loss: 0.4449 - val_acc: 0.7931 - val_f1_score: 0.8738
Epoch 3/5
172/172 [==============================] - 89s 518ms/step - loss: 0.4406 - acc: 0.7932 - f1_score: 0.8744 - val_loss: 0.4670 - val_acc: 0.7660 - val_f1_score: 0.8598
Epoch 4/5
172/172 [==============================] - 89s 516ms/step - loss: 0.4339 - acc: 0.7883 - f1_score: 0.8702 - val_loss: 0.4629 - val_acc: 0.7716 - val_f1_score: 0.8602
Epoch 5/5
172/172 [==============================] - 89s 518ms/step - loss: 533584.6875 - acc: 0.7732 - f1_score: 0.8607 - val_loss: 1.3926 - val_acc: 0.7459 - val_f1_score: 0.8443


In [ ]:
#without dropout + clean text ------------ resturant
NN.fit(train_text, y_train, validation_data=(val_text, y_val), epochs=5, batch_size=50, verbose=1)

Epoch 1/5
172/172 [==============================] - 57s 332ms/step - loss: 292.6279 - acc: 0.7621 - f1_score: 0.8618 - val_loss: 0.4804 - val_acc: 0.7623 - val_f1_score: 0.8628
Epoch 2/5
172/172 [==============================] - 57s 329ms/step - loss: 0.4330 - acc: 0.7987 - f1_score: 0.8779 - val_loss: 0.4916 - val_acc: 0.8178 - val_f1_score: 0.8839
Epoch 3/5
172/172 [==============================] - 57s 331ms/step - loss: 0.3914 - acc: 0.8286 - f1_score: 0.8922 - val_loss: 0.4066 - val_acc: 0.8234 - val_f1_score: 0.8882
Epoch 4/5
172/172 [==============================] - 57s 333ms/step - loss: 0.3618 - acc: 0.8396 - f1_score: 0.8987 - val_loss: 0.4006 - val_acc: 0.8305 - val_f1_score: 0.8910
Epoch 5/5
172/172 [==============================] - 56s 326ms/step - loss: 0.3842 - acc: 0.8351 - f1_score: 0.8959 - val_loss: 0.4007 - val_acc: 0.8258 - val_f1_score: 0.8917


In [ ]:
#without dropout + clean text ------------ resturant
NN.fit(train_text, y_train, validation_data=(val_text, y_val), epochs=20, batch_size=100, verbose=1)

Epoch 1/20
86/86 [==============================] - 50s 581ms/step - loss: 0.1300 - acc: 0.9513 - f1_score: 0.9679 - val_loss: 0.5318 - val_acc: 0.8431 - val_f1_score: 0.8984
Epoch 2/20
86/86 [==============================] - 46s 531ms/step - loss: 0.1104 - acc: 0.9594 - f1_score: 0.9732 - val_loss: 0.5912 - val_acc: 0.8267 - val_f1_score: 0.8846
Epoch 3/20
86/86 [==============================] - 44s 517ms/step - loss: 0.0999 - acc: 0.9616 - f1_score: 0.9745 - val_loss: 0.6988 - val_acc: 0.8473 - val_f1_score: 0.9028
Epoch 4/20
86/86 [==============================] - 46s 538ms/step - loss: 0.0940 - acc: 0.9649 - f1_score: 0.9768 - val_loss: 0.6057 - val_acc: 0.8417 - val_f1_score: 0.8976
Epoch 5/20
86/86 [==============================] - 46s 540ms/step - loss: 0.4939 - acc: 0.9150 - f1_score: 0.9445 - val_loss: 4.5749 - val_acc: 0.7482 - val_f1_score: 0.8530
Epoch 6/20
86/86 [==============================] - 46s 537ms/step - loss: 7269897.0000 - acc: 0.7025 - f1_score: 0.8178 - va

In [ ]:
#without dropout + clean text + added dense layer ------------ resturant
NN.fit(train_text, y_train, validation_data=(val_text, y_val), epochs=5, batch_size=50, verbose=1)

Epoch 1/5
172/172 [==============================] - 143s 815ms/step - loss: 299186.5000 - acc: 0.7345 - f1_score: 0.8401 - val_loss: 0.5122 - val_acc: 0.7520 - val_f1_score: 0.8577
Epoch 2/5
172/172 [==============================] - 59s 343ms/step - loss: 0.4972 - acc: 0.7526 - f1_score: 0.8574 - val_loss: 0.5033 - val_acc: 0.7525 - val_f1_score: 0.8571
Epoch 3/5
172/172 [==============================] - 59s 341ms/step - loss: 0.4880 - acc: 0.7574 - f1_score: 0.8595 - val_loss: 0.5001 - val_acc: 0.7506 - val_f1_score: 0.8557
Epoch 4/5
172/172 [==============================] - 56s 326ms/step - loss: 0.4828 - acc: 0.7575 - f1_score: 0.8590 - val_loss: 0.4971 - val_acc: 0.7506 - val_f1_score: 0.8535
Epoch 5/5
172/172 [==============================] - 58s 340ms/step - loss: 0.4784 - acc: 0.7621 - f1_score: 0.8607 - val_loss: 0.4950 - val_acc: 0.7515 - val_f1_score: 0.8540


In [ ]:
#without dropout + clean str (few preprocess than clean text) ------------ resturant
NN.fit(train_text, y_train, validation_data=(val_text, y_val), epochs=10, batch_size=50, verbose=1)

Epoch 1/10
172/172 [==============================] - 56s 302ms/step - loss: 23.0487 - acc: 0.7456 - f1_score: 0.8482 - val_loss: 0.5044 - val_acc: 0.7501 - val_f1_score: 0.8572
Epoch 2/10
172/172 [==============================] - 53s 308ms/step - loss: 0.7080 - acc: 0.7658 - f1_score: 0.8641 - val_loss: 0.4766 - val_acc: 0.7679 - val_f1_score: 0.8645
Epoch 3/10
172/172 [==============================] - 51s 297ms/step - loss: 0.4507 - acc: 0.7814 - f1_score: 0.8703 - val_loss: 0.4590 - val_acc: 0.7837 - val_f1_score: 0.8695
Epoch 4/10
172/172 [==============================] - 53s 311ms/step - loss: 0.4346 - acc: 0.7936 - f1_score: 0.8752 - val_loss: 0.4452 - val_acc: 0.7940 - val_f1_score: 0.8748
Epoch 5/10
172/172 [==============================] - 51s 298ms/step - loss: 2449.4436 - acc: 0.7958 - f1_score: 0.8752 - val_loss: 0.4362 - val_acc: 0.8010 - val_f1_score: 0.8780
Epoch 6/10
172/172 [==============================] - 53s 308ms/step - loss: 0.4656 - acc: 0.7806 - f1_score: 0

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

seq_len = 128 # standardized length of each word sequence 

#------------------------------------------------------ Choose a dataset df
#df= res_df
df= prod_df
#df= htl_df
#df= mov_df

#################################### vectorizing cleaned text
# fit tokenizer vocab 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.clean_text)

################################### standard train/val split
train_text, val_text, y_train, y_val = train_test_split(df.clean_text, df.polarity, 
                                                        test_size=0.2, random_state=123, stratify=df.polarity)


################################## indexing + padding
# convert train and val texts to token sequences of standardized length 128,
# padding fills leading 0s in or cuts off sequence at 128th word
train_text = tokenizer.texts_to_sequences(train_text) 
train_text = pad_sequences(train_text, maxlen=seq_len, padding='post')

val_text = tokenizer.texts_to_sequences(val_text)
val_text = pad_sequences(val_text, maxlen=seq_len, padding='post')

print('one training text after padding: ')
print(train_text[10])

################################### preparing embedding matrix
embedding_dim = w2v.vector_size # w2v embedding dim
word_index = tokenizer.word_index # vocab

# use the gensim model to build a numpy array of embeddings,
# we'll feed this array to the keras embeddings layer.
# each row i of the array will correspond to the word token assigned to that value 
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    try:
        embedding_vector = w2v[word]
        embedding_matrix[i] = embedding_vector
    except: # word in our data vocab is missing in w2v, will use 0 vector for that word
        pass
print('embedding matrix shape (vocab, embeddings dim) is ', embedding_matrix.shape)


##################################### defining model

inp = Input(shape=(seq_len,))
x = Embedding(len(word_index) + 1,
              embedding_dim,
              weights=[embedding_matrix], # where we feed the pretrained vecs
              trainable=False)(inp) # freeze these parameters in the model
#recurrent_dropout=.1
x = Bidirectional(LSTM(64, activation='relu'))(x)
x = Dense(32)(x) # fully connected layer on top of the output of the bi-LSTM
#x = Dropout(.3)(x)
#x = Dense(20)(x) #added
y = Dense(1, activation='sigmoid')(x)

NN = Model(inp, y)

print(NN.summary())

######################################## compiling model
threshold = 0.5
NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',tfa.metrics.F1Score(num_classes=2, average='micro', threshold=threshold)])


######################################## fitting model = training
NN.fit(train_text, y_train, validation_data=(val_text, y_val), epochs=10, batch_size=50, verbose=1)


######################################### testing

# This can be used if we have train, val, test sets
# but since data size is not to large, I choose 2 splits only
#result = the validation of last epoch

prob= NN.predict(val_text)
predections= [1 if p >= 0.5 else 0 for p in prob]
print('---------------------------RESULT------------------------')
print('The model f1 score on validation set is :', f1_score(y_val, predections))


one training text after padding: 
[   5 4911 4912    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
embedding matrix shape (vocab, embeddings dim) is  (9908, 300)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 128, 300)          2972400   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              186880    
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 32)                4128      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,163,441
Trainable params: 191,041
Non-trainable params: 2,972,400
___________________________________________

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

seq_len = 128 # standardized length of each word sequence 

#------------------------------------------------------ Choose a dataset df
#df= res_df
#df= prod_df
df= htl_df
#df= mov_df

#################################### vectorizing cleaned text
# fit tokenizer vocab 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.clean_text)

################################### standard train/val split
train_text, val_text, y_train, y_val = train_test_split(df.clean_text, df.polarity, 
                                                        test_size=0.2, random_state=123, stratify=df.polarity)


################################## indexing + padding
# convert train and val texts to token sequences of standardized length 128,
# padding fills leading 0s in or cuts off sequence at 128th word
train_text = tokenizer.texts_to_sequences(train_text) 
train_text = pad_sequences(train_text, maxlen=seq_len, padding='post')

val_text = tokenizer.texts_to_sequences(val_text)
val_text = pad_sequences(val_text, maxlen=seq_len, padding='post')

print('one training text after padding: ')
print(train_text[10])

################################### preparing embedding matrix
embedding_dim = w2v.vector_size # w2v embedding dim
word_index = tokenizer.word_index # vocab

# use the gensim model to build a numpy array of embeddings,
# we'll feed this array to the keras embeddings layer.
# each row i of the array will correspond to the word token assigned to that value 
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    try:
        embedding_vector = w2v[word]
        embedding_matrix[i] = embedding_vector
    except: # word in our data vocab is missing in w2v, will use 0 vector for that word
        pass
print('embedding matrix shape (vocab, embeddings dim) is ', embedding_matrix.shape)


##################################### defining model

inp = Input(shape=(seq_len,))
x = Embedding(len(word_index) + 1,
              embedding_dim,
              weights=[embedding_matrix], # where we feed the pretrained vecs
              trainable=False)(inp) # freeze these parameters in the model
#recurrent_dropout=.1
x = Bidirectional(LSTM(64, activation='relu'))(x)
x = Dense(32)(x) # fully connected layer on top of the output of the bi-LSTM
#x = Dropout(.3)(x)
#x = Dense(20)(x) #added
y = Dense(1, activation='sigmoid')(x)

NN = Model(inp, y)

print(NN.summary())

######################################## compiling model
threshold = 0.5
NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',tfa.metrics.F1Score(num_classes=2, average='micro', threshold=threshold)])


######################################## fitting model = training
NN.fit(train_text, y_train, validation_data=(val_text, y_val), epochs=10, batch_size=50, verbose=1)


######################################### testing

# This can be used if we have train, val, test sets
# but since data size is not to large, I choose 2 splits only
#result = the validation of last epoch

prob= NN.predict(val_text)
predections= [1 if p >= 0.5 else 0 for p in prob]
print('---------------------------RESULT------------------------')
print('The model f1 score on validation set is :', f1_score(y_val, predections))


one training text after padding: 
[73645   669    65 16068 24665   354  1963 25444  9015   681 37482 37483
 14973  2873  2968    11    47  2264  1432  5626   438   782     7 17589
 12493  5977 15135  2909 12146   920   258    78   979 20174   776   125
    12   696   942  4167    54     1 13347    82   235  1445   391  1574
   415   301     1   208   802   965    45 73646    60  3204   871   728
 34468 19787   432     8   924  1264 73647     1  2172  7351 21030 16417
 22121  7327  3070  6356  5940    10  2035   691  5166   162    29 26612
   180  3493  4637   348    24   582  2714  3006 73648 16353  1101 73649
   188   166  1459   162   288  2536   620     3  1442   976    34   385
  5595     7  5626  2264  6539 26347    13   813 31132   317     1  1631
 13556   586   592 36829 12184  3822    19    23]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


embedding matrix shape (vocab, embeddings dim) is  (85463, 300)
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 128, 300)          25638900  
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              186880    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 32)                4128      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 25,829,941
Trainable params: 191,041
Non-traina

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

seq_len = 128 # standardized length of each word sequence 

#------------------------------------------------------ Choose a dataset df
#df= res_df
#df= prod_df
#df= htl_df
df= mov_df

#################################### vectorizing cleaned text
# fit tokenizer vocab 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.clean_text)

################################### standard train/val split
train_text, val_text, y_train, y_val = train_test_split(df.clean_text, df.polarity, 
                                                        test_size=0.2, random_state=123, stratify=df.polarity)


################################## indexing + padding
# convert train and val texts to token sequences of standardized length 128,
# padding fills leading 0s in or cuts off sequence at 128th word
train_text = tokenizer.texts_to_sequences(train_text) 
train_text = pad_sequences(train_text, maxlen=seq_len, padding='post')

val_text = tokenizer.texts_to_sequences(val_text)
val_text = pad_sequences(val_text, maxlen=seq_len, padding='post')

print('one training text after padding: ')
print(train_text[10])

################################### preparing embedding matrix
embedding_dim = w2v.vector_size # w2v embedding dim
word_index = tokenizer.word_index # vocab

# use the gensim model to build a numpy array of embeddings,
# we'll feed this array to the keras embeddings layer.
# each row i of the array will correspond to the word token assigned to that value 
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    try:
        embedding_vector = w2v[word]
        embedding_matrix[i] = embedding_vector
    except: # word in our data vocab is missing in w2v, will use 0 vector for that word
        pass
print('embedding matrix shape (vocab, embeddings dim) is ', embedding_matrix.shape)


##################################### defining model

inp = Input(shape=(seq_len,))
x = Embedding(len(word_index) + 1,
              embedding_dim,
              weights=[embedding_matrix], # where we feed the pretrained vecs
              trainable=False)(inp) # freeze these parameters in the model
#recurrent_dropout=.1
x = Bidirectional(LSTM(64, activation='relu'))(x)
x = Dense(32)(x) # fully connected layer on top of the output of the bi-LSTM
#x = Dropout(.3)(x)
#x = Dense(20)(x) #added
y = Dense(1, activation='sigmoid')(x)

NN = Model(inp, y)

print(NN.summary())

######################################## compiling model
threshold = 0.5
NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',tfa.metrics.F1Score(num_classes=2, average='micro', threshold=threshold)])


######################################## fitting model = training
NN.fit(train_text, y_train, validation_data=(val_text, y_val), epochs=10, batch_size=50, verbose=1)


######################################### testing

# This can be used if we have train, val, test sets
# but since data size is not to large, I choose 2 splits only
#result = the validation of last epoch

prob= NN.predict(val_text)
predections= [1 if p >= 0.5 else 0 for p in prob]
print('---------------------------RESULT------------------------')
print('The model f1 score on validation set is :', f1_score(y_val, predections))


one training text after padding: 
[ 5578  7055   263  6755   125   890    71  6265    22  6799  1160 29361
   603    79    49  1314    53   327  2713   403  2690  2415    15  1230
 29362    89   673 14053  4009  2796  2090 13427 14054  2090  3698 14055
 18734   398   731   689  1668   164  2351    58 18735     7  3719    32
   100  6806  1990  2791 29363    18    32  1277 29364 29365 18736  1045
    58  5629 29366  1912     7    42   204   284 11267 18737 14056 14057
 11268  6266    22  2284   329 18643  5130  7971   982    40    22  9028
   303   505    49  4674    81 29367  3475 29368   372     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


embedding matrix shape (vocab, embeddings dim) is  (62508, 300)
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128)]             0         
                                                                 
 embedding_3 (Embedding)     (None, 128, 300)          18752400  
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              186880    
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 32)                4128      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 18,943,441
Trainable params: 191,041
Non-traina

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

seq_len = 128 # standardized length of each word sequence 

#------------------------------------------------------ Choose a dataset df
#df= res_df
#df= prod_df
#df= htl_df
#df= mov_df

df= pd.concat([res_df,prod_df,htl_df,mov_df])

#################################### vectorizing cleaned text
# fit tokenizer vocab 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.clean_text)

################################### standard train/val split
train_text, val_text, y_train, y_val = train_test_split(df.clean_text, df.polarity, 
                                                        test_size=0.2, random_state=123, stratify=df.polarity)


################################## indexing + padding
# convert train and val texts to token sequences of standardized length 128,
# padding fills leading 0s in or cuts off sequence at 128th word
train_text = tokenizer.texts_to_sequences(train_text) 
train_text = pad_sequences(train_text, maxlen=seq_len, padding='post')

val_text = tokenizer.texts_to_sequences(val_text)
val_text = pad_sequences(val_text, maxlen=seq_len, padding='post')

print('one training text after padding: ')
print(train_text[10])

################################### preparing embedding matrix
embedding_dim = w2v.vector_size # w2v embedding dim
word_index = tokenizer.word_index # vocab

# use the gensim model to build a numpy array of embeddings,
# we'll feed this array to the keras embeddings layer.
# each row i of the array will correspond to the word token assigned to that value 
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    try:
        embedding_vector = w2v[word]
        embedding_matrix[i] = embedding_vector
    except: # word in our data vocab is missing in w2v, will use 0 vector for that word
        pass
print('embedding matrix shape (vocab, embeddings dim) is ', embedding_matrix.shape)


##################################### defining model

inp = Input(shape=(seq_len,))
x = Embedding(len(word_index) + 1,
              embedding_dim,
              weights=[embedding_matrix], # where we feed the pretrained vecs
              trainable=False)(inp) # freeze these parameters in the model
#recurrent_dropout=.1
x = Bidirectional(LSTM(64, activation='relu'))(x)
x = Dense(32)(x) # fully connected layer on top of the output of the bi-LSTM
#x = Dropout(.3)(x)
#x = Dense(20)(x) #added
y = Dense(1, activation='sigmoid')(x)

NN = Model(inp, y)

print(NN.summary())

######################################## compiling model
threshold = 0.5
NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',tfa.metrics.F1Score(num_classes=2, average='micro', threshold=threshold)])


######################################## fitting model = training
NN.fit(train_text, y_train, validation_data=(val_text, y_val), epochs=10, batch_size=50, verbose=1)


######################################### testing

# This can be used if we have train, val, test sets
# but since data size is not to large, I choose 2 splits only
#result = the validation of last epoch

prob= NN.predict(val_text)
predections= [1 if p >= 0.5 else 0 for p in prob]
print('---------------------------RESULT------------------------')
print('The model f1 score on validation set is :', f1_score(y_val, predections))


one training text after padding: 
[    91   4921    961   2039    227    224    353  17643  44581     91
    401   2813     85      6  60782    220    131   6108     11    241
    371    467     51    614   3501   2989     55     78    182   1699
     36      6  35705 114340   2042   9754   9779  20073      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


embedding matrix shape (vocab, embeddings dim) is  (147742, 300)
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128)]             0         
                                                                 
 embedding_4 (Embedding)     (None, 128, 300)          44322600  
                                                                 
 bidirectional_4 (Bidirectio  (None, 128)              186880    
 nal)                                                            
                                                                 
 dense_8 (Dense)             (None, 32)                4128      
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 44,513,641
Trainable params: 191,041
Non-train

**ArSenTD-Lev (Arabic Sentiment Twitter Dataset for LEVantine dialect)**

For this dataset, I will use w2v pretrained on Twitter tweets.
Since the dataset is small, I will not consider single domains.

In [23]:
!wget 'https://bakrianoo.ewr1.vultrobjects.com/aravec/full_uni_sg_300_twitter.zip'
!unzip 'full_uni_sg_300_twitter.zip'

--2021-12-15 17:00:51--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_uni_sg_300_twitter.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2832094149 (2.6G) [application/zip]
Saving to: ‘full_uni_sg_300_twitter.zip’

full_uni_sg_300_twi 100%[===================>]   2.64G  47.8MB/s    in 38s     

2021-12-15 17:01:29 (71.2 MB/s) - ‘full_uni_sg_300_twitter.zip’ saved [2832094149/2832094149]

Archive:  full_uni_sg_300_twitter.zip
  inflating: full_uni_sg_300_twitter.mdl  
  inflating: full_uni_sg_300_twitter.mdl.trainables.syn1neg.npy  
  inflating: full_uni_sg_300_twitter.mdl.wv.vectors.npy  


In [24]:
# load the AraVec model
tw2v = gensim.models.Word2Vec.load("full_uni_sg_300_twitter.mdl")
print("We've",len(tw2v.wv.index2word),"vocabularies") 

We've 1259756 vocabularies


In [25]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

In [26]:
# define 'clean_tweet' function to clean the text and remove unwanted text parts
def clean_tweet(text):
    # define regular expression patterns
    p_english = "[a-zA-Z0-9]+"
    p_url = "https?://[A-Za-z0-9./]+"
    p_mention = "\@[\_0-9a-zA-Z]+\:?"    
    p_retweet = "RT \@[\_\-0-9a-zA-Z]+\:?"
    p_punctuations = "[" + string.punctuation + "]"
    
    # remove unwanted parts
    text = re.sub(p_english, ' ', text)
    text = re.sub(p_retweet, ' ', text)
    text = re.sub(p_mention, ' ', text)
    text = re.sub(p_url, ' ', text)
    text = re.sub(p_punctuations, ' ', text)
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    
    # remove الهمزة
    text = re.sub("[أإآ]", 'ا', text)
    text = re.sub("ة", 'ه', text)
    text = re.sub("ى", 'ي', text)
    
    # removing tashkeel
    tashkel = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(tashkel, '', text)
    
    # remove repeated letters more than two letters
    text = re.sub(r'(.)\1+', r'\1\1', text)
    text = text.strip()

    text= remove_emoji(text)
    
    # remove stopwords
    words = [word for word in text.split() if word not in ar_stopwords]
    words = [word for word in words if len(word)>=2]
    
    # merge and return final text
    return ' '.join(words)

In [28]:
#read files of first dataset (MSA)
cor_df=pd.read_csv('ArSenTD-LEV.tsv', sep='\t')
cor_df= cor_df.drop(columns=['Country', 'Sentiment_Expression','Sentiment_Target'])
cor_df=cor_df[cor_df['Sentiment']!='neutral'].reset_index(drop=True)
#unify format with other dataset
cor_df['Sentiment'].replace({'very_positive': '1', 'positive': '1', 'very_negative':'0', 'negative':'0'}, inplace=True)
cor_df= cor_df.reset_index(drop=True)
cor_df.rename(columns={'Tweet': 'text', 'Sentiment': 'polarity'}, inplace=True)
cor_df["polarity"] = pd.to_numeric(cor_df["polarity"])
cor_df['clean_text'] = cor_df['text'].apply(clean_tweet)
cor_df

,text,Topic,polarity,clean_text
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",personal,0,اؤمن بان الانسان ينطفئ جماله ابتعاد يحب بريق ا...
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,sports,1,الذاكره اعتقد كريستيانو افضل لاعب العالم كاكا ...
2,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,personal,0,مصطلحات لبنانيه حيرت البشريه بتوصل عالبيت بنط ...
3,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,personal,0,نصمت لتسير حياتنا يرام فالناس تعد نقيه
4,@Yousef_MUFC اكثر ما يزعجنا بعد مستوانا خارج ا...,sports,0,يزعجنا مستوانا خارج ارضنا تمثيل كونتي فوزه
...,...,...,...,...
3110,نهتم من خلال خدمة تنسيق الرسائل بإظهار رسالة ا...,education,1,نهتم خدمه تنسيق الرسائل باظهار رساله الماجستير...
3111,صلاح من لاعب في المقاولون العرب يحلم ان يلعب ل...,sports,1,صلاح لاعب المقاولون العرب يحلم يلعب للاهلي وال...
3112,الملك سلمان بن عبد العزيز: تطبيق الأنظمة بحزم ...,politics,1,الملك سلمان عبد العزيز تطبيق الانظمه بحزم تطاو...
3113,@ZahraaIraq9 😂 كل ما ادخل حسابي الكه تغريداتج ...,personal,0,ادخل حسابي الكه تغريداتج حب العراق وانتي هسه ي...


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

seq_len = 128 # standardized length of each word sequence 


df= cor_df

#################################### vectorizing cleaned text
# fit tokenizer vocab 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.clean_text)

################################### standard train/val split
train_text, val_text, y_train, y_val = train_test_split(df.clean_text, df.polarity, 
                                                        test_size=0.2, random_state=123, stratify=df.polarity)


################################## indexing + padding
# convert train and val texts to token sequences of standardized length 128,
# padding fills leading 0s in or cuts off sequence at 128th word
train_text = tokenizer.texts_to_sequences(train_text) 
train_text = pad_sequences(train_text, maxlen=seq_len, padding='post')

val_text = tokenizer.texts_to_sequences(val_text)
val_text = pad_sequences(val_text, maxlen=seq_len, padding='post')

print('one training text after padding: ')
print(train_text[10])

################################### preparing embedding matrix
embedding_dim = tw2v.vector_size # w2v embedding dim
word_index = tokenizer.word_index # vocab

# use the gensim model to build a numpy array of embeddings,
# we'll feed this array to the keras embeddings layer.
# each row i of the array will correspond to the word token assigned to that value 
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    try:
        embedding_vector = tw2v[word]
        embedding_matrix[i] = embedding_vector
    except: # word in our data vocab is missing in w2v, will use 0 vector for that word
        pass
print('embedding matrix shape (vocab, embeddings dim) is ', embedding_matrix.shape)


##################################### defining model

inp = Input(shape=(seq_len,))
x = Embedding(len(word_index) + 1,
              embedding_dim,
              weights=[embedding_matrix], # where we feed the pretrained vecs
              trainable=False)(inp) # freeze these parameters in the model
#recurrent_dropout=.1
x = Bidirectional(LSTM(64, activation='relu'))(x)
x = Dense(32)(x) # fully connected layer on top of the output of the bi-LSTM
#x = Dropout(.3)(x)
#x = Dense(20)(x) #added
y = Dense(1, activation='sigmoid')(x)

NN = Model(inp, y)

print(NN.summary())

######################################## compiling model
threshold = 0.5
NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',tfa.metrics.F1Score(num_classes=2, average='micro', threshold=threshold)])


######################################## fitting model = training
NN.fit(train_text, y_train, validation_data=(val_text, y_val), epochs=10, batch_size=50, verbose=1)


######################################### testing

# This can be used if we have train, val, test sets
# but since data size is not to large, I choose 2 splits only
#result = the validation of last epoch

prob= NN.predict(val_text)
predections= [1 if p >= 0.5 else 0 for p in prob]
print('---------------------------RESULT------------------------')
print('The model f1 score on validation set is :', f1_score(y_val, predections))


one training text after padding: 
[   43    43    43    76   412 16479  1436    38 16480 16481 16482 16483
 16484  4526 16485 16486   446     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


embedding matrix shape (vocab, embeddings dim) is  (17055, 300)
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 128)]             0         
                                                                 
 embedding_5 (Embedding)     (None, 128, 300)          5116500   
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              186880    
 nal)                                                            
                                                                 
 dense_10 (Dense)            (None, 32)                4128      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 5,307,541
Trainable params: 191,041
Non-trainab